# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install proj

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geoviews

In [3]:
pip install cartopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
geoapify_key

'94f922f2b4294d1ba8e951ccb2ff4ff3'

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khairpur,27.5295,68.7617,30.94,27,0,2.13,PK,1730433140
1,1,namibe,-15.1961,12.1522,16.87,84,96,1.81,AO,1730433142
2,2,port-aux-francais,-49.3500,70.2167,4.09,84,100,11.52,TF,1730433143
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.91,87,62,9.39,SH,1730433144
4,4,sao gabriel da cachoeira,-0.1303,-67.0892,25.09,70,61,1.13,BR,1730433145


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,mermoz boabab,14.7065,-17.4758,25.88,94,0,2.06,SN,1730433236
97,97,oak island,33.9166,-78.1611,22.23,86,0,1.54,US,1730433264
151,151,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.03,GF,1730433328
189,189,hadibu,12.6500,54.0333,25.93,70,0,3.50,YE,1730433376
322,322,san patricio,28.0170,-97.5169,25.83,80,0,2.91,US,1730433550
339,339,tamiahua,21.2667,-97.4500,25.06,93,0,2.50,MX,1730433573
385,385,saint-pierre,-21.3393,55.4781,25.82,50,0,2.57,RE,1730433635
416,416,sharm el-sheikh,27.8518,34.3050,22.08,53,0,2.57,EG,1730433700
506,506,khuzdar,27.7384,66.6434,22.49,18,0,2.31,PK,1730433881
510,510,hilton head island,32.2163,-80.7526,21.11,79,0,0.00,US,1730433885


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\jerem\AppData\Local\Temp\ipykernel_2584\964000501.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
77,mermoz boabab,SN,14.7065,-17.4758,94,
97,oak island,US,33.9166,-78.1611,86,
151,remire-montjoly,GF,4.9167,-52.2667,89,
189,hadibu,YE,12.6500,54.0333,70,
322,san patricio,US,28.0170,-97.5169,80,
339,tamiahua,MX,21.2667,-97.4500,93,
385,saint-pierre,RE,-21.3393,55.4781,50,
416,sharm el-sheikh,EG,27.8518,34.3050,53,
506,khuzdar,PK,27.7384,66.6434,18,
510,hilton head island,US,32.2163,-80.7526,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apikey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longtitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longtitude}, {latitude}, {radius}"
    params["bias"] = f"proximity: {longtitude}, {latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mermoz boabab - nearest hotel: No hotel found
oak island - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
tamiahua - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
sharm el-sheikh - nearest hotel: No hotel found
khuzdar - nearest hotel: No hotel found
hilton head island - nearest hotel: No hotel found
longboat key - nearest hotel: No hotel found
kasane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
77,mermoz boabab,SN,14.7065,-17.4758,94,No hotel found
97,oak island,US,33.9166,-78.1611,86,No hotel found
151,remire-montjoly,GF,4.9167,-52.2667,89,No hotel found
189,hadibu,YE,12.6500,54.0333,70,No hotel found
322,san patricio,US,28.0170,-97.5169,80,No hotel found
339,tamiahua,MX,21.2667,-97.4500,93,No hotel found
385,saint-pierre,RE,-21.3393,55.4781,50,No hotel found
416,sharm el-sheikh,EG,27.8518,34.3050,53,No hotel found
506,khuzdar,PK,27.7384,66.6434,18,No hotel found
510,hilton head island,US,32.2163,-80.7526,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)